In [1]:
from coir.data_loader import get_tasks
from coir.evaluation import COIR
from coir.models import YourCustomDEModel
from coir.beir.retrieval import TrainRetriever
import pandas as pd

/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/envs/virtual-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
expl_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CoSQA_explanations_query_code.csv'
expl_df = pd.read_csv(expl_path)
expl_df.rename(columns={"query_id": "query-id", "corpus_id": "corpus-id"}, inplace=True)
expl_df.head()

,Unnamed: 0,query-id,doc,corpus-id,code,explanation_granite_1,explanation_granite_2,explanation_granite_3,explanation_granite_4,explanation_deepseek_1_cleaned,explanation_deepseek_2_cleaned,explanation_deepseek_3_cleaned,explanation_deepseek_4_cleaned
0,0,q1,python code to write bool value 1,d1,"def writeBoolean(self, n):\n """"""\n ...",\nThe provided code is a Python method named `...,"\n1. `def writeBoolean(self, n):` - This line ...",\nThis code snippet is a function that writes ...,\nThe code snippet provided is a method named ...,The doc string and code both write the boolean...,"Alright, I need to break down this Python code...",The code snippet writes a boolean value to a s...,The code snippet defines a method called write...
1,1,q2,"""python how to manipulate clipboard""",d2,"def paste(xsel=False):\n """"""Returns system ...",\nThe provided Python code snippet defines a f...,\n1. `def paste(xsel=False):` - This line defi...,\nThis code snippet is a function in Python th...,\nThe code snippet provided is a Python functi...,The paste function pastes data from the clipbo...,The code snippet begins by defining a function...,The code snippet is a function called paste th...,The code snippet defines a function called pas...
2,2,q3,python colored output to html,d3,"def _format_json(data, theme):\n """"""Pretty ...","\nThis Python function, `_format_json`, takes ...","\n1. `def _format_json(data, theme):` - This l...","\nThis code snippet, named `_format_json`, is ...",\n```python\nimport json\nimport sys\nimport p...,The docstring explains that the function _form...,"```pythondef _format_json(data, theme): """"""...",This code snippet is a function called _format...,To explain the code snippet in a way that can ...
3,3,q4,"python ""create directory"" using ""relative path""",d4,"def create_path(path):\n """"""Creates a absol...","\nThe provided Python function, `create_path(p...",\n1. The code snippet defines a function named...,\nThis code snippet helps you create a new dir...,\nThe provided code snippet is a Python functi...,The doc and code create an absolute path from ...,"Okay, I need to explain this Python code snipp...",This code creates a file system where any path...,The code snippet defines a function called cre...
4,4,q5,python column of an array,d5,"def _vector_or_scalar(x, type='row'):\n """"""...",\nThe provided Python code defines a function ...,\nThis code snippet defines a function called ...,\nThis code snippet is a function that convert...,\nThe code snippet provided is a function defi...,The code checks if an object `x` is an array o...,"Alright, so I've got this Python code snippet ...",The code snippet is a Python function called _...,"Okay, I'm going to try to figure out how to ex..."


In [4]:
tasks = get_tasks(tasks=["cosqa"],segment="train")

in tasks 
fetched data from hf
data loader init


20604it [00:00, 180237.26it/s]


In [ ]:
corpus, queries, qrels = tasks['cosqa']

for _, row in expl_df.iterrows():
    corpus_id = row['corpus-id']
    explanation = row['explanation_deepseek_1_cleaned']

    if corpus_id in corpus and explanation and explanation.strip():
        corpus[corpus_id]['text'] = explanation  

corpus = {doc_id: doc for doc_id, doc in corpus.items() if 'text' in doc and doc['text'].strip()}
print(corpus['d1'])
tasks['cosqa'] = (corpus, queries, qrels)

print(f"Total docs in corpus after replacement: {len(tasks['cosqa'][0])}")
missing_text = sum(1 for doc in tasks['cosqa'][0].values() if not doc.get('text'))
print(f"Number of docs still missing 'text': {missing_text}")

In [ ]:
train_retriever = TrainRetriever(model_name, batch_size)
train_samples = train_retriever.load_train(corpus_data, query_data, qrels_data)
train_dataloader = train_retriever.prepare_train(train_samples)
train_retriever.fit(train_objectives=train_dataloader)

# Now load the fine-tuned model
model = YourCustomDEModel(model_name="path_to_finetuned_model")
evaluation = COIR(tasks=tasks, batch_size=128)
results = evaluation.run(model, output_folder=f"results/{model_name}_explanation_deepseek_1")
